In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

In [2]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=2,
                          num_workers=10,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=2,
                          num_workers=10,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=2,
                          num_workers=10,
                          shuffle=True)

In [3]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.zeros_(m.bias.data)
        
epochs     = 100
criterion = nn.CrossEntropyLoss()
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()
                labels = Y.cuda()
            else:
                inputs, labels = X.cpu(), Y.cpu()

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

       # val(epoch)
        fcn_model.train()


def val(epoch):
    # Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    p_acc = 0
    iou_acc = 0
    count = 0
    for iter, (X, tar, Y) in enumerate(val_loader):
        if use_gpu:
            X = X.cuda()
            tar = tar.cuda()
        else:
            X,tar = X.cpu(), tar.cpu()
        p, iou = fcn_model.eval(X, tar)
        p_acc += p
        iou_acc += np.mean(np.array(iou))
        count += 1
    print("Epoch {}: Pixel Acc: {}, IOU Acc: {}".format(epoch, p_acc/count, iou_acc/count))
    return p_acc/count, iou_acc/count
        
    
    
def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    p_acc = 0
    iou_acc = 0
    count = 0
    for iter, (X, tar, Y) in enumerate(test_loader):
        if use_gpu:
            X = X.cuda()
            tar = tar.cuda()
        else:
            X,tar = X.cpu(), tar.cpu()
        p, iou = fcn_model.eval(X, tar)
        p_acc += p
        iou_acc += np.mean(np.array(iou))
        count += 1
    print("Pixel Acc: {}, IOU Acc: {}".format(p_acc/count, iou_acc/count))
    return p_acc/count, iou_acc/count
    
if __name__ == "__main__":
#    val(0)# show the accuracy before training
#     test()
    train()
    print("Testing ")
    test()

epoch0, iter0, loss: 3.8715436458587646
epoch0, iter10, loss: 3.7951738834381104
epoch0, iter20, loss: 3.086334228515625
epoch0, iter30, loss: 3.341083288192749
epoch0, iter40, loss: 2.636950731277466
epoch0, iter50, loss: 2.951385498046875
epoch0, iter60, loss: 2.223762273788452
epoch0, iter70, loss: 2.3895506858825684
epoch0, iter80, loss: 2.007445812225342
epoch0, iter90, loss: 2.418191432952881
epoch0, iter100, loss: 1.9197317361831665
epoch0, iter110, loss: 1.78251051902771
epoch0, iter120, loss: 2.420910120010376
epoch0, iter130, loss: 2.1020991802215576
epoch0, iter140, loss: 2.272207498550415
epoch0, iter150, loss: 2.0991053581237793
epoch0, iter160, loss: 1.8706785440444946
epoch0, iter170, loss: 1.9488638639450073
epoch0, iter180, loss: 1.9733035564422607
epoch0, iter190, loss: 2.1032755374908447
epoch0, iter200, loss: 1.8109874725341797
epoch0, iter210, loss: 2.3548665046691895
epoch0, iter220, loss: 1.850690245628357
epoch0, iter230, loss: 1.723262906074524
epoch0, iter240,

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FCN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/s

epoch1, iter0, loss: 1.6564420461654663
epoch1, iter10, loss: 1.8193912506103516
epoch1, iter20, loss: 2.035170316696167
epoch1, iter30, loss: 1.8300786018371582
epoch1, iter40, loss: 1.6798810958862305
epoch1, iter50, loss: 2.0562057495117188
epoch1, iter60, loss: 1.4898600578308105
epoch1, iter70, loss: 2.377103567123413
epoch1, iter80, loss: 1.6682374477386475
epoch1, iter90, loss: 2.1490232944488525
epoch1, iter100, loss: 1.74490487575531
epoch1, iter110, loss: 2.2052507400512695
epoch1, iter120, loss: 1.802778959274292
epoch1, iter130, loss: 1.4630740880966187
epoch1, iter140, loss: 1.5818285942077637
epoch1, iter150, loss: 1.675787091255188
epoch1, iter160, loss: 1.882788896560669
epoch1, iter170, loss: 1.800306797027588
epoch1, iter180, loss: 1.9284648895263672
epoch1, iter190, loss: 1.5219966173171997
epoch1, iter200, loss: 1.5665562152862549
epoch1, iter210, loss: 1.5346462726593018
epoch1, iter220, loss: 1.703981876373291
epoch1, iter230, loss: 1.6497042179107666
epoch1, iter

epoch2, iter460, loss: 1.8868125677108765
epoch2, iter470, loss: 1.6216063499450684
epoch2, iter480, loss: 1.6048641204833984
epoch2, iter490, loss: 1.677466630935669
epoch2, iter500, loss: 1.526831030845642
epoch2, iter510, loss: 1.8589203357696533
epoch2, iter520, loss: 2.2212531566619873
epoch2, iter530, loss: 1.8737143278121948
epoch2, iter540, loss: 1.842675805091858
epoch2, iter550, loss: 1.5368947982788086
epoch2, iter560, loss: 1.5371012687683105
epoch2, iter570, loss: 1.8014966249465942
epoch2, iter580, loss: 1.6093683242797852
epoch2, iter590, loss: 1.4400399923324585
epoch2, iter600, loss: 1.520333170890808
epoch2, iter610, loss: 1.4740945100784302
epoch2, iter620, loss: 2.2055087089538574
epoch2, iter630, loss: 1.6500779390335083
epoch2, iter640, loss: 1.9225389957427979
epoch2, iter650, loss: 2.061208486557007
epoch2, iter660, loss: 1.659845232963562
epoch2, iter670, loss: 1.4839911460876465
epoch2, iter680, loss: 1.6921876668930054
epoch2, iter690, loss: 1.401025176048278

epoch3, iter920, loss: 1.427917718887329
epoch3, iter930, loss: 1.6495444774627686
epoch3, iter940, loss: 1.7003211975097656
epoch3, iter950, loss: 2.408477783203125
epoch3, iter960, loss: 1.6190952062606812
epoch3, iter970, loss: 1.9659489393234253
epoch3, iter980, loss: 1.4056135416030884
epoch3, iter990, loss: 1.4195770025253296
epoch3, iter1000, loss: 1.76118004322052
epoch3, iter1010, loss: 1.6335748434066772
epoch3, iter1020, loss: 1.630658507347107
epoch3, iter1030, loss: 1.660638451576233
epoch3, iter1040, loss: 1.383533239364624
epoch3, iter1050, loss: 1.9561563730239868
epoch3, iter1060, loss: 1.5029433965682983
epoch3, iter1070, loss: 1.944451928138733
epoch3, iter1080, loss: 1.3983933925628662
epoch3, iter1090, loss: 1.4319387674331665
epoch3, iter1100, loss: 1.573784589767456
epoch3, iter1110, loss: 1.561848521232605
epoch3, iter1120, loss: 2.110987901687622
epoch3, iter1130, loss: 1.33143949508667
epoch3, iter1140, loss: 1.607954978942871
epoch3, iter1150, loss: 2.2190840

...

epoch94, iter160, loss: 1.347174048423767
epoch94, iter170, loss: 1.3153486251831055
epoch94, iter180, loss: 1.5564303398132324
epoch94, iter190, loss: 1.408629298210144
epoch94, iter200, loss: 1.9884923696517944
epoch94, iter210, loss: 1.2919254302978516
epoch94, iter220, loss: 1.3064357042312622
epoch94, iter230, loss: 1.6691099405288696
epoch94, iter240, loss: 1.761760950088501
epoch94, iter250, loss: 1.28269362449646
epoch94, iter260, loss: 1.3679028749465942
epoch94, iter270, loss: 1.3817005157470703
epoch94, iter280, loss: 1.0467153787612915
epoch94, iter290, loss: 1.3395472764968872
epoch94, iter300, loss: 1.2563376426696777
epoch94, iter310, loss: 1.694738745689392
epoch94, iter320, loss: 1.403059959411621
epoch94, iter330, loss: 1.6476815938949585
epoch94, iter340, loss: 1.3710861206054688
epoch94, iter350, loss: 1.7018922567367554
epoch94, iter360, loss: 1.6160649061203003
epoch94, iter370, loss: 1.268847107887268
epoch94, iter380, loss: 1.4140089750289917
epoch94, iter390, l

epoch95, iter570, loss: 1.5025062561035156
epoch95, iter580, loss: 1.3093737363815308
epoch95, iter590, loss: 1.1109727621078491
epoch95, iter600, loss: 1.503090262413025
epoch95, iter610, loss: 1.669156789779663
epoch95, iter620, loss: 1.3909835815429688
epoch95, iter630, loss: 1.571628212928772
epoch95, iter640, loss: 1.1612695455551147
epoch95, iter650, loss: 1.3254382610321045
epoch95, iter660, loss: 1.485558271408081
epoch95, iter670, loss: 2.077073097229004
epoch95, iter680, loss: 1.6700674295425415
epoch95, iter690, loss: 1.5596166849136353
epoch95, iter700, loss: 1.307283878326416
epoch95, iter710, loss: 1.429896593093872
epoch95, iter720, loss: 1.679215431213379
epoch95, iter730, loss: 1.4032047986984253
epoch95, iter740, loss: 1.3458759784698486
epoch95, iter750, loss: 1.3411948680877686
epoch95, iter760, loss: 1.5815575122833252
epoch95, iter770, loss: 1.6037198305130005
epoch95, iter780, loss: 1.2727431058883667
epoch95, iter790, loss: 2.116673231124878
epoch95, iter800, lo

epoch96, iter980, loss: 1.391703486442566
epoch96, iter990, loss: 1.3983683586120605
epoch96, iter1000, loss: 1.9743114709854126
epoch96, iter1010, loss: 1.7769858837127686
epoch96, iter1020, loss: 1.2167291641235352
epoch96, iter1030, loss: 1.4307307004928589
epoch96, iter1040, loss: 1.523762583732605
epoch96, iter1050, loss: 1.2731999158859253
epoch96, iter1060, loss: 1.8417837619781494
epoch96, iter1070, loss: 1.4772531986236572
epoch96, iter1080, loss: 1.3963078260421753
epoch96, iter1090, loss: 1.3534237146377563
epoch96, iter1100, loss: 1.6219875812530518
epoch96, iter1110, loss: 1.5546735525131226
epoch96, iter1120, loss: 1.3484917879104614
epoch96, iter1130, loss: 1.4261606931686401
epoch96, iter1140, loss: 1.4535084962844849
epoch96, iter1150, loss: 1.6283031702041626
epoch96, iter1160, loss: 1.8952434062957764
epoch96, iter1170, loss: 1.400223970413208
epoch96, iter1180, loss: 1.299728512763977
epoch96, iter1190, loss: 1.5520923137664795
epoch96, iter1200, loss: 1.08546137809

epoch97, iter1380, loss: 1.1501035690307617
epoch97, iter1390, loss: 1.364645004272461
epoch97, iter1400, loss: 1.6632771492004395
epoch97, iter1410, loss: 1.3635810613632202
epoch97, iter1420, loss: 1.5435386896133423
epoch97, iter1430, loss: 1.2037092447280884
epoch97, iter1440, loss: 1.5533002614974976
epoch97, iter1450, loss: 1.6616296768188477
epoch97, iter1460, loss: 1.5205903053283691
epoch97, iter1470, loss: 1.5561970472335815
epoch97, iter1480, loss: 1.5761311054229736
Finish epoch 97, time elapsed 600.4126811027527
epoch98, iter0, loss: 1.4738123416900635
epoch98, iter10, loss: 1.3479994535446167
epoch98, iter20, loss: 1.215152621269226
epoch98, iter30, loss: 1.4224200248718262
epoch98, iter40, loss: 1.3521838188171387
epoch98, iter50, loss: 1.2777118682861328
epoch98, iter60, loss: 1.6768087148666382
epoch98, iter70, loss: 1.3628770112991333
epoch98, iter80, loss: 1.8035333156585693
epoch98, iter90, loss: 1.7091387510299683
epoch98, iter100, loss: 1.5467174053192139
epoch98,

epoch99, iter290, loss: 1.1968830823898315
epoch99, iter300, loss: 1.348116397857666
epoch99, iter310, loss: 1.448935627937317
epoch99, iter320, loss: 1.1655938625335693
epoch99, iter330, loss: 1.2178000211715698
epoch99, iter340, loss: 1.379102349281311
epoch99, iter350, loss: 1.3607935905456543
epoch99, iter360, loss: 1.1068735122680664
epoch99, iter370, loss: 1.6017060279846191
epoch99, iter380, loss: 1.3853164911270142
epoch99, iter390, loss: 2.156423568725586
epoch99, iter400, loss: 1.311525583267212
epoch99, iter410, loss: 1.366147756576538
epoch99, iter420, loss: 1.3398510217666626
epoch99, iter430, loss: 1.4226884841918945
epoch99, iter440, loss: 1.771802544593811
epoch99, iter450, loss: 1.1816471815109253
epoch99, iter460, loss: 1.2410681247711182
epoch99, iter470, loss: 1.2084050178527832
epoch99, iter480, loss: 1.431637167930603
epoch99, iter490, loss: 1.6037462949752808
epoch99, iter500, loss: 1.41013503074646
epoch99, iter510, loss: 1.4711517095565796
epoch99, iter520, los